# Task 3

# Table of Contents
- [Task 3.1](#Task 3.1)
- [Task 3.2](#Task 3.2)
- [Task 3.3](#Task 3.3)



In [81]:
import numpy as np
import pandas as pd
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy
import random as r
from scipy import ndimage
from scipy.misc import toimage

%matplotlib inline

In [82]:
#Read images
from skimage.io import imread
im_cameraman = imread("HW1/Images/Q1/cameraman.jpg")
im_house = imread("HW1/Images/Q1/house.jpg")
im_lena = imread("HW1/Images/Q1/lena.jpg")

#Convert to grayscale by using average of RGB
def average(pixel):
    return (int(pixel[0])+int(pixel[1])+int(pixel[2]))/3

def rgb2gray(image):
    gray = np.zeros((image.shape[0], image.shape[1])) # init 2D numpy array
    for rownum in range(len(image)):
        for colnum in range(len(image[rownum])):
            gray[rownum][colnum] = average(image[rownum][colnum])

    return gray

im_cameraman_gray = rgb2gray(im_cameraman)
im_house_gray = rgb2gray(im_house)
im_lena_gray = rgb2gray(im_lena)

# Task 3.1
<a id='Task 3.1'></a>

In [1]:
from skimage import feature
from skimage import filters

def apply_sobel(image,pixel_value):
    edges = filters.sobel(image)
    edges = global_threshold(edges,pixel_value)
    
    return edges

def global_threshold(image,pixel_value):
    
    for rownum in range(len(image)):
        for colnum in range(len(image[rownum])):
            if image[rownum][colnum] > pixel_value:
                image[rownum][colnum] = 255
            else:
                image[rownum][colnum] = 0
                

    return image


def apply_gaussian_laplace(image,sigma,threshold): 
    
    # Skiimage.ndimage.filters
    edges = ndimage.filters.gaussian_laplace(image,sigma)
    
    # zero crossing + local thresholding
    edges_after = zero_crossing_local_thresholding(edges,threshold)
    
    return edges_after

def zero_crossing_local_thresholding(edges,threshold): 
    sol_image = np.zeros((edges.shape[0], edges.shape[1])) # init 2D numpy array
    
    #1. They should be done in a 3x3 block for every 3x3 block (apart from the borders of the image)
    for rownum in range(len(edges)-2):
        for colnum in range(len(edges[rownum])-2):
            temp_3by3 = np.matrix([
                                  [edges[rownum][colnum],edges[rownum][colnum+1],edges[rownum][colnum+2]],
                                  [edges[rownum+1][colnum],edges[rownum+1][colnum+1],edges[rownum+1][colnum+2]],
                                  [edges[rownum+2][colnum],edges[rownum+2][colnum+1],edges[rownum+2][colnum+2]]])
            
            #2. Zero is crossed when the minimum and the maximum of the block have different signs.
            #3. If they have the same sign, or one of them is zero, zero was not crossed.
            #4. A pixel passes the threshold if the difference between the maximum and
            #   the minimum (MAX - MIN) in its 3x3 block is bigger than the threshold value.
    
            is_zero_crossed = is_crossed(temp_3by3,threshold)
            
            sol_image[rownum+1][colnum+1] = is_zero_crossed
            
    return sol_image

def is_crossed(matrix,threshold):
    min_val = np.min(matrix)
    

    max_val = np.max(matrix)
    if (max_val-min_val < threshold) and ((min_val > 0 and max_val > 0) or (min_val <0 and max_val<0)):
        return 0 #zero is not crossed
    else:
        return 1 #zero is crossed



def apply_canny(image,sigma):
    
    # Compute the Canny filter for a value of sigma
    edges = feature.canny(image, sigma)
    return edges

def show_canny_only(edges):
    
    # display results
    fig, (ax2) = plt.subplots(nrows=1, ncols=1, figsize=(8, 3),
                                        sharex=True, sharey=True)

    ax2.imshow(edges1, cmap=plt.cm.gray)
    ax2.axis('off')
    ax2.set_title('Canny filter, $\sigma=5$', fontsize=20)

    fig.tight_layout()

    plt.show()
    pass
    
    

    

In [2]:
plt.imshow(apply_gaussian_laplace(im_cameraman_gray,2,5),cmap=plt.cm.gray)

NameError: name 'plt' is not defined

In [79]:

m_middle = np.matrix([[2,7,7],
                      [5,7,3],
                      [4,7,1]])
is_crossed(m_middle)

1
7


0

In [59]:
#e1 = apply_canny(im_cameraman_gray)
#e2 = apply_canny(im_house_gray)
#e3 = apply_canny(im_lena_gray)
#e4 = apply_sobel(im_cameraman_gray)
e4 = apply_gaussian_laplace(im_cameraman_gray,1)
print e4
df = pd.DataFrame(e4)
df

[[ 1.0974791  -0.34302262 -1.83492597 ...,  0.32523453 -0.03597575
  -0.13209403]
 [ 0.89452417  0.26131776 -0.56692498 ..., -0.01501969 -0.07075436
  -0.03333878]
 [-0.5616828   0.30787766  0.92071728 ..., -0.57012134 -0.06965358
   0.08305569]
 ..., 
 [ 1.99402977 -0.69998612 -0.37908188 ...,  1.32933072  3.81288345
   3.99287248]
 [ 1.89945887  0.30411069  0.83783664 ...,  0.14854672  2.46803231
   3.02351016]
 [ 0.88719883  0.76398263  0.95394682 ...,  0.39672419  3.06781437
   3.20892221]]


,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,1.097479,-0.343023,-1.834926,-2.066888,-0.592729,1.232500,1.424156,0.362879,-0.665561,0.166080,...,0.176280,0.346035,-0.456850,0.271700,1.360151,1.162487,0.622676,0.325235,-0.035976,-0.132094
1,0.894524,0.261318,-0.566925,-1.117336,-0.655490,0.375287,0.620242,0.188173,-0.404722,-0.007581,...,0.211201,0.306602,0.087683,0.821414,1.335102,0.748337,0.246112,-0.015020,-0.070754,-0.033339
2,-0.561683,0.307878,0.920717,0.550645,-0.072953,-0.203250,-0.274062,-0.177507,-0.183595,-0.430396,...,-0.287448,-0.573594,-0.584799,-0.014090,0.187230,-0.536351,-0.848982,-0.570121,-0.069654,0.083056
3,-1.841212,0.019086,1.443032,1.357756,0.621109,0.114914,-0.270640,-0.215903,-0.099565,-0.424014,...,-0.204573,-0.738341,-1.072877,-0.717636,-0.450831,-1.031146,-1.248757,-0.564424,0.083529,0.158821
4,-1.158633,0.008817,0.676872,0.411263,0.129299,0.226761,0.496806,0.566541,0.156332,0.073914,...,0.191908,-0.026008,-0.234929,0.224150,0.587274,0.194132,-0.242726,0.113656,0.371787,0.281465
5,-0.013854,0.108893,-0.196439,-1.017117,-1.231269,-0.332991,1.089406,1.514081,0.432778,0.189082,...,-0.282873,0.086970,0.407046,1.151802,1.492727,1.109524,0.436970,0.330288,0.325655,0.213762
6,-0.873058,-0.175649,0.224593,-0.551976,-1.253723,-0.652756,0.663910,1.232072,0.327183,-0.251732,...,-0.647499,-0.253436,-0.354257,0.196498,0.658591,0.261728,-0.381491,-0.482929,-0.307721,-0.288401
7,-2.335023,-0.503928,1.320275,1.165251,-0.085418,-0.603111,-0.246551,0.124109,-0.146413,-0.573235,...,-0.485035,-0.638718,-1.420059,-1.107637,-0.452253,-0.581790,-0.912869,-1.069340,-0.689834,-0.315356
8,-1.833161,-0.208176,1.705056,1.879343,0.595555,-0.643316,-0.924254,-0.762157,-0.732639,-0.544159,...,-0.152342,-1.087396,-2.003175,-1.944302,-1.137894,-0.178061,0.188503,-0.408849,-0.496241,0.008182
9,-0.378348,0.262843,1.309446,1.665892,0.872869,-0.519826,-1.131918,-0.957408,-0.666248,-0.181187,...,0.897110,0.198263,-0.814278,-1.756456,-1.764010,-0.369512,0.757323,0.514426,-0.026105,-0.067926
